In [1]:
import sys
sys.path.insert(0, '/home1/dthoang/idx2/build/Source/Python')
#sys.path.insert(0, 'F:/Workspace/idx2/build/Source/Python/RelWithDebInfo')
#sys.path.insert(0, 'F:/Workspace/idx2/build/Source/Python/Debug')
import idx2Py as i2p
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from ipywidgets import *
import concurrent.futures

In [2]:
%%time
input_dir  = '/nobackupp19/vpascucc/converted_files/nasa-encoding-framework'
accuracy = 0.01
#vmin = -6.14770936965942382812
#vmax = 1.24035537242889404297
vmin = -5.35
vmax = 3.24

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [3]:
import colorsys

Points = [
  -5.35081,
  -0.732399,
  -0.214108,
  0.00697509,
  0.210571,
  0.424882,
  3.24308  
]

RGBPoints = [  
  [0.27843137254900002, 0.27843137254900002, 0.85882352941200002],
  [0.0, 0.33333333333333331, 1.0],
  [0.33333333333333331, 0.66666666666666663, 1.0],
  [0.0, 0.0, 0.49803921568627452],  
  [0.82745098039215681, 1.0, 0.73333333333333328],
  [1.0, 0.66666666666666663, 0.0],
  [0.87843137254899994, 0.30196078431399997, 0.30196078431399997]
]

def get_continuous_cmap(rgb_list, float_list=None):
  if float_list:
    # normalize the float list
    min_val = min(float_list)
    max_val = max(float_list)
    my_range = max_val - min_val
    float_list = [(v - min_val)/my_range for v in float_list]
    print(float_list)
  else:
    float_list = list(np.linspace(0, 1, len(rgb_list)))

  cdict = dict()  
  for num, col in enumerate(['red', 'green', 'blue']):
    col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(0, len(float_list))]
    cdict[col] = col_list
  cmp = matplotlib.colors.LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=256)
  return cmp

my_cmap = get_continuous_cmap(RGBPoints, Points)

[0.0, 0.5374063433439339, 0.5977155863060849, 0.6234411995033681, 0.6471319739954782, 0.6720695750120144, 1.0]


# Visualize a single face through time

In [6]:
%%time
%matplotlib inline

downsampling = [2, 2, 2]
array = i2p.Decode3f32('/nobackupp19/vpascucc/converted_files/nasa-encoding-framework/llc2160/u-face-1-depth-3-time-0-1024.idx2', input_dir, downsampling, accuracy)

plot = None
def f(t): # get time step t
  fig = plt.figure(figsize = (4, 12))
  plot = plt.imshow(array[t, :, :], origin = 'upper', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax)
  plt.show()
  return

interact(f, t = widgets.IntSlider(min = 0, max = array.shape[0] - 1, step = 1, value = 0, layout = Layout(width = '1200px')));


interactive(children=(IntSlider(value=0, description='t', layout=Layout(width='1200px'), max=256), Output()), …

CPU times: user 16.6 s, sys: 1.61 s, total: 18.2 s
Wall time: 18.5 s


<function __main__.f(t)>

# Visualize four faces through time

In [12]:
%%time
%matplotlib inline

downsampling = [2, 2, 2]

array0 = None
array1 = None
array3 = None
array4 = None

with concurrent.futures.ThreadPoolExecutor() as executor:
    accuracy = 0.01
    future0 = executor.submit(i2p.Decode3f32, '/nobackupp19/vpascucc/converted_files/nasa-encoding-framework/llc2160/u-face-0-depth-0-time-0-1024.idx2', input_dir, downsampling, accuracy);
    future1 = executor.submit(i2p.Decode3f32, '/nobackupp19/vpascucc/converted_files/nasa-encoding-framework/llc2160/u-face-1-depth-0-time-0-1024.idx2', input_dir, downsampling, accuracy);
    future3 = executor.submit(i2p.Decode3f32, '/nobackupp19/vpascucc/converted_files/nasa-encoding-framework/llc2160/u-face-3-depth-0-time-0-1024.idx2', input_dir, downsampling, accuracy);
    future4 = executor.submit(i2p.Decode3f32, '/nobackupp19/vpascucc/converted_files/nasa-encoding-framework/llc2160/u-face-4-depth-0-time-0-1024.idx2', input_dir, downsampling, accuracy);
    array0 = future0.result()
    array1 = future1.result()
    array3 = future3.result()
    array4 = future4.result()


# vmin = min(np.min(array0), np.min(array1), np.min(array3), np.min(array4))
# vmax = max(np.max(array0), np.max(array1), np.max(array3), np.max(array4))

def f(t): # get time step t
  fig, ((ax0, ax1, ax3, ax4)) = plt.subplots(1, 4, figsize=([15, 12]))
  ax0.axis('off')
  ax1.axis('off')
  ax3.axis('off')
  ax4.axis('off')

  ax0.imshow(array0[t, :, :], origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax)
  ax1.imshow(array1[t, :, :], origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax)
  ax3.imshow(np.rot90(array3[t, :, :]), origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax)
  ax4.imshow(np.rot90(array4[t, :, :]), origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax) 
  return  

interact(f, t = widgets.IntSlider(min = 0, max = array1.shape[0] - 1, step = 1, value = 0, layout = Layout(width = '1200px')));

interactive(children=(IntSlider(value=0, description='t', layout=Layout(width='1200px'), max=256), Output()), …

CPU times: user 1min 10s, sys: 7.44 s, total: 1min 17s
Wall time: 21.5 s


<function __main__.f(t)>

# Visualize four faces through depths

In [10]:
%%time
%matplotlib inline

array0 = None
array1 = None
array3 = None
array4 = None

downsampling = [3, 3, 3]

def f(d): # get time step t
  with concurrent.futures.ThreadPoolExecutor() as executor:
    accuracy = 0.01
    future0 = executor.submit(i2p.Decode3f32, 'D:/Datasets/nasa/llc_2160_32/llc2160/u-face-0-depth-' + repr(d) + '-time-0-32.idx2', input_dir, downsampling, accuracy)    
    future1 = executor.submit(i2p.Decode3f32, 'D:/Datasets/nasa/llc_2160_32/llc2160/u-face-1-depth-' + repr(d) + '-time-0-32.idx2', input_dir, downsampling, accuracy)
    future3 = executor.submit(i2p.Decode3f32, 'D:/Datasets/nasa/llc_2160_32/llc2160/u-face-3-depth-' + repr(d) + '-time-0-32.idx2', input_dir, downsampling, accuracy)
    future4 = executor.submit(i2p.Decode3f32, 'D:/Datasets/nasa/llc_2160_32/llc2160/u-face-4-depth-' + repr(d) + '-time-0-32.idx2', input_dir, downsampling, accuracy)
    array0 = future0.result()
    array1 = future1.result()
    array3 = future3.result()
    array4 = future4.result()
    fig, ((ax0, ax1, ax3, ax4)) = plt.subplots(1, 4, figsize=([15, 12]))
    ax0.axis('off')
    ax1.axis('off')
    ax3.axis('off')
    ax4.axis('off')
    ax0.imshow(array0[0, :, :], origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax)
    ax1.imshow(array1[0, :, :], origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax)
    ax3.imshow(np.rot90(array3[0, :, :]), origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax)
    ax4.imshow(np.rot90(array4[0, :, :]), origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax) 
  return  

interact(f, d = widgets.IntSlider(min = 0, max = 89, step = 1, value = 0, layout = Layout(width = '800px')));

interactive(children=(IntSlider(value=0, description='d', layout=Layout(width='800px'), max=89), Output()), _d…

CPU times: total: 2.98 s
Wall time: 1.32 s


<function __main__.f(d)>

# Vertical slicing along y (constant x) at a particular time step

In [5]:
%%time
%matplotlib inline

downsampling = [2, 2, 2]

timestep = 0
ndepths = 90
n = 2160

def to_face(f):
  if (f >= 2):
    return f + 1;
  return f
    
def f(x):
  face = to_face(x // n)
  xx = x % n # local x coordinate within the face
  nx = 0
  ny = ndepths
  future_array = [None] * ndepths
  with concurrent.futures.ThreadPoolExecutor() as executor:
    for d in range(0, ndepths):  
      extent = (xx, 0, timestep, 1, n * 3, 1)    
      if (face > 2):
        extent = (0, xx, timestep, n * 3, 1, 1)
      #print(extent)
      #print('D:/Datasets/nasa/llc_2160_32/llc2160/u-face-' + repr(face) + '-depth-' + repr(d) + '-time-0-32.idx2')
      future_array[d] = executor.submit(i2p.DecodeExtent3f32, 'D:/Datasets/nasa/llc_2160_32/llc2160/u-face-' + repr(face) + '-depth-' + repr(d) + '-time-0-32.idx2', input_dir, extent, downsampling, accuracy)

  array = None # store the whole result
  for d in range(0, ndepths):
    # figure out the shape of the output array once (at depth 0)
    a = future_array[d].result()
    #if (face > 2):
      #a = np.rot90(a)
    if d == 0:
      print(a.shape)
      if (face < 2):
        nx = a.shape[1]
      else:
        nx = a.shape[2]
      array = np.empty((ny, nx))
      
    if (face < 2):
      array[d, :] = a[0, :, 0]
    else:
      array[d, :] = a[0, 0, :]
    #array[d, :] = a[0, :, (xx * a.shape[2]) // n]

  fig = plt.plot(figsize = ([1024, 90]))
  plt.imshow(array, origin = 'lower', cmap = my_cmap, aspect = 'auto', vmin = vmin, vmax = vmax)
  plt.show()
  return

interact(f, x = widgets.IntSlider(min = 0, max = 8639, step = 10, value = 0, layout = Layout(width = '800px')));

interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='800px'), max=8639, step=10), Ou…

CPU times: total: 37.5 s
Wall time: 4.17 s


<function __main__.f(x)>